https://medium.com/data-science-journal/how-to-measure-feature-importance-in-a-binary-classification-model-d284b8c9a301

In [37]:
# Loading 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json

# 1 -- Load Data

# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]
    
all_data = pd.concat(data, ignore_index=True, sort=True)

In [46]:
basic_features = ['participant', 'session_num', 'engagement']
binary_features = []
numerical_features = []
for c in all_data.columns:
    if 'AU' in c or '_change' in c or '_skill' in c or '_diff' in c or c in ['of_success', 'ros_in_game']:
        binary_features.append(c)
    elif c not in basic_features:
        numerical_features.append(c)

In [50]:
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [51]:
for c in all_data.columns:
    if c in binary_features or c in numerical_features:
        mean = np.nanmean(all_data[c])
        std = np.nanstd(all_data[c])

        if std == 0:
            all_data[c] = (all_data[c]-mean)
        else:
            all_data[c] = (all_data[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(all_data[c])

            all_data[c] = all_data[c].fillna(min_val)
        else:
            max_val = np.nanmax(all_data[c])

            all_data[c] = all_data[c].fillna(max_val)

In [57]:
from scipy import stats
import operator

In [62]:
numerical_results = {}
for f in numerical_features:
    numerical_results[f] = abs(stats.pearsonr(all_data[f], all_data['engagement'])[0])
numerical_results = sorted(numerical_results.items(), key=operator.itemgetter(1))

In [65]:
for i in numerical_results:
    print(i[1])

0.0014502406304759195
0.0017782615758854593
0.004986697348104668
0.005505187489160796
0.005535664990880406
0.006642256498713
0.008704788901455263
0.008925163261466997
0.011607145762594872
0.011846459436433759
0.012725962990906545
0.013051646787529955
0.013359094117668403
0.014423763149933738
0.016288489630288974
0.016593175555371313
0.02158991723482078
0.02241784996667637
0.03117951430559799
0.03535912183520486
0.03561208790469342
0.03818247663904372
0.043780482277260925
0.04614711930166213
0.046249419719211586
0.04787854860987493
0.05478053105594041
0.05811506532712101
0.06160013355198118
0.06210537327724743
0.06531728881981687
0.06980548429634395
0.07587516966960571
0.078865865138507
0.08878430389420519
0.09242877915845539
0.1499570404700644
0.1520311120256648
0.1702729225164488
0.1720009357835782
0.20455640578748924
0.25353648320357375
0.2822739245251549
0.28260394024542
0.30060157180072444
0.3386475397526343
0.34930952578855634
0.37507106235256465
0.3781938190372459
0.3801401788528

In [71]:
categorical_results = {}
for f in binary_features:
    val = all_data[f].unique()[0]
    sub1 = all_data.loc[all_data[f]==val]
    sub2 = all_data.loc[all_data[f]!=val]
    
    e1 = sub1['engagement'].sum()/len(sub1)
    e2 = sub2['engagement'].sum()/len(sub2)
    
    categorical_results[f] = abs(e1-e2)
categorical_results = sorted(categorical_results.items(), key=operator.itemgetter(1))

In [74]:
for i in categorical_results:
    print(i[1])

0.0036842821561401395
0.004041405251665187
0.012387165432729441
0.030506898280247108
0.031005788040396
0.03181390652030214
0.03503464104380816
0.03748800028844379
0.04158878348373185
0.06313456042934273
0.07852469077357904
0.07925149167698764
0.07926061779549065
0.08287085439547426
0.08367141555701685
0.08965011227089525
0.09462759928992692
0.09541962184815644
0.09853584758850586
0.10142323758648786
0.10260054334261381
0.12859325362390006
0.13031566171877873
0.153730146296164
0.1564683180165658
0.16118302468599865
0.1623058772444923
0.164414300501594
0.16659229763964745
0.1704411414459196
0.17571016707012999
0.17845174090260774
0.18362092926482265
0.1837503730024378
0.1846791945706756
0.19036456871846763
0.1910270161529345
0.19463392683807612
0.19592058162544057
0.19822563941020355
0.19919503243529035
0.20027416723011482
0.20096147179045432
0.20605811251607586
0.21314342949062826
0.22632535787604102
0.35926351306010684
0.4068834585587272


In [75]:
key_features = [
'timestamp',
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked',]

In [77]:
for f in key_features:
    for o in key_features:
        if f != o:
            print(f, o, stats.pearsonr(all_data[f], all_data[o])[0])
    print()

op_num_people of_pose_distance -0.4189830930901057
op_num_people of_gaze_distance -0.3931370608537672
op_num_people of_confidence 0.3988818093419966
op_num_people timestamp -0.29731584806344935
op_num_people ros_mistakes_session -0.2968238075471269
op_num_people ros_ts_robot_talked -0.24685979945763195
op_num_people ros_ts_game_start -0.16449935660980225

of_pose_distance op_num_people -0.4189830930901057
of_pose_distance of_gaze_distance 0.9928299847175869
of_pose_distance of_confidence -0.967234226424457
of_pose_distance timestamp 0.25161344802350566
of_pose_distance ros_mistakes_session 0.25294065032769614
of_pose_distance ros_ts_robot_talked 0.18766005035438976
of_pose_distance ros_ts_game_start 0.15519280671566812

of_gaze_distance op_num_people -0.3931370608537672
of_gaze_distance of_pose_distance 0.9928299847175869
of_gaze_distance of_confidence -0.9748452282304991
of_gaze_distance timestamp 0.24017227098481295
of_gaze_distance ros_mistakes_session 0.23991382212907755
of_gaze_di